# Emittance measurement simulation

In [1]:
import sys
import os
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm

sys.path.append('/Users/46h/Research/')
from scdist.tools import beam_analysis as ba
from scdist.tools import plotting as myplt
from scdist.tools import utils

In [2]:
pplt.rc['grid.alpha'] = 0.05
pplt.rc['axes.grid'] = False
pplt.rc['figure.facecolor'] = 'white'
format_kws = dict(ygrid=True, xgrid=False)
savefig_kws = dict(dpi=300)

## Load data

In [3]:
turns = np.loadtxt('_output/data/turns.dat').astype(int)
print(turns)

frames = list(range(len(turns)))

[  0  50 100 150 200 250 300 350]


In [4]:
intensities = np.loadtxt('_output/data/intensities.dat')
print(intensities)

[1.50000000e+11 7.86428571e+12 1.55785714e+13 2.32928571e+13
 3.10071429e+13 3.87214286e+13 4.64357143e+13 5.41500000e+13]


In [5]:
file = open('_output/data/rec_node_names.txt', 'r')
rec_node_names = [line.rstrip() for line in file]
file.close()

In [6]:
rec_node_positions = np.loadtxt('_output/data/rec_node_positions.dat')

In [7]:
istart = 35
rec_node_names = rec_node_names[istart:]
rec_node_positions = rec_node_positions[istart:]

Sigmas_rec[i, j, :, :] is the reconstructed covariance matrix $\Sigma$ for node i, trial j. We also compute the mean and standard deviation of $\Sigma$ across trials. We use the mean values to compute the beam statistics.

In [8]:
Sigmas_rec_list = []
for frame in frames:
    Sigmas = np.load('_output/data/Sigmas_rec_{}.npy'.format(frame))
    Sigmas = Sigmas[istart:]
    Sigmas_rec_list.append(Sigmas)

Now load the simulated moments at each node.

In [9]:
Sigmas_sim_list = []
for frame in frames:
    Sigmas = np.load('_output/data/Sigmas_sim_{}.npy'.format(frame))
    Sigmas = Sigmas[istart:]
    Sigmas_sim_list.append(Sigmas)
    
stats_sim_list = []
for Sigmas in Sigmas_sim_list:
    moments = np.array([utils.mat2vec(Sigma) for Sigma in Sigmas])
    stats = ba.BeamStats()
    stats.read_moments(moments)
    stats_sim_list.append(stats)

## Comparison 

In [10]:
line_kws_sim = dict(lw=1.0)
line_kws_rec = dict(ls='dotted', lw=1.0)

In [11]:
# for i, frame in enumerate(frames):
#     stats_rec = stats_rec_list[i]
#     stats_sim = stats_sim_list[i]
#     turn = turns[i]
    
#     fig, axes = pplt.subplots(nrows=2, figsize=(5.5, 3.75))
#     g1 = axes[0].plot(rec_node_positions[:, 0], stats_sim.twiss2D['beta_x'], **line_kws_sim)
#     g2 = axes[0].plot(rec_node_positions[:, 0], stats_sim.twiss2D['beta_y'], **line_kws_sim)
#     axes[0].format(cycle='colorblind')
#     g3 = axes[0].plot(rec_node_positions[:, 0], stats_rec.twiss2D['beta_x'], **line_kws_rec)
#     g4 = axes[0].plot(rec_node_positions[:, 0], stats_rec.twiss2D['beta_y'], **line_kws_rec)
#     axes[0].legend(
#         [g1, g2, g3, g4], 
#         labels=[r'$\beta_x$ (sim)', r'$\beta_y$ (sim)', 
#                 r'$\beta_x$ (rec)', r'$\beta_y$ (rec)'],
#         ncols=1, loc='r', framealpha=0.
#     )
#     g1 = axes[1].plot(rec_node_positions[:, 0], stats_sim.twiss2D['alpha_x'], **line_kws_sim)
#     g2 = axes[1].plot(rec_node_positions[:, 0], stats_sim.twiss2D['alpha_y'], **line_kws_sim)
#     axes[1].format(cycle='colorblind')
#     g3 = axes[1].plot(rec_node_positions[:, 0], stats_rec.twiss2D['alpha_x'], **line_kws_rec)
#     g4 = axes[1].plot(rec_node_positions[:, 0], stats_rec.twiss2D['alpha_y'], **line_kws_rec)
#     axes[1].legend(
#         [g1, g2, g3, g4], 
#         labels=[r'$\alpha_x$ (sim)', r'$\alpha_x$ (sim)', 
#                 r'$\alpha_x$ (rec)', r'$\alpha_y$ (rec)'],
#         ncols=1, loc='r', framealpha=0.
#     )
#     axes[0].format(
#         ylabel='[m/rad]', xlabel='Position [m]', 
#         ylim=(0., 105.),
#         title='Reconstructed Twiss parameters -- turn {}'.format(turn)
#     )
#     axes[1].format(
#         ylabel='[rad]', xlabel='Position [m]', 
#         ylim=(-max(axes[1].get_ylim()), max(axes[1].get_ylim()))
#     )
#     plt.show()

In [12]:
sim_kws = dict(color='black')
rec_kws = dict(color='red', lw=0, marker='.', ms=2)
# rec_kws = dict(color='red', ls='dotted')
labels = [
    r"$\langle{xx}\rangle$",
    r"$\langle{xx'}\rangle$",
    r"$\langle{xy}\rangle$",
    r"$\langle{xy'}\rangle$",
    r"$\langle{x'x'}\rangle$",
    r"$\langle{x'y}\rangle$",
    r"$\langle{x'y'}\rangle$",
    r"$\langle{yy}\rangle$",
    r"$\langle{yy'}\rangle$",
    r"$\langle{y'y'}\rangle$",
]

In [13]:
def mean_std(Sigmas, func):
    data = [func(Sigma) for Sigma in Sigmas]
    return np.mean(data, axis=0), np.std(data, axis=0)

In [14]:
def means_stds(Sigmas_at_nodes, func):
    means, stds = [], []
    for Sigmas in Sigmas_at_nodes:
        mean, std = mean_std(Sigmas, func)
        means.append(mean)
        stds.append(std)
    means = np.array(means)
    stds = np.array(stds)
    return means, stds

In [ ]:
_frames = [1, 3, 5, 7]
ebar_kws = dict(elinewidth=1.0, ecolor='red', capsize=0, lw=0, zorder=999)

fig, axes = pplt.subplots(nrows=10, ncols=len(_frames), figsize=(7, 10.5), spany=False, aligny=True)

for j, frame in enumerate(_frames):        
    Sigmas_rec = Sigmas_rec_list[frame]
    means, stds = means_stds(Sigmas_rec, ba.mat2vec)  
    sims = np.array([ba.mat2vec(Sigma) for Sigma in Sigmas_sim_list[frame]])    
    for i, ax in enumerate(axes[:, j]):
        ax.plot(rec_node_positions[:, 0], sims[:, i], **sim_kws)
        ax.errorbar(rec_node_positions[:, 0], means[:, i], yerr=stds[:, i], **ebar_kws)

for i, ax in enumerate(axes[:, 0]):
    ax.set_ylabel(labels[i])
for frame, ax in zip(_frames, axes[0, :]):
    ax.set_title('Turn {}'.format(turns[frame]))
xmin, xmax = axes[-1].get_xlim()
axes.format(xlim=(xmin, 1.02 * xmax))
axes.format(suptitle='Second-order moments', xlabel='Position [m]')
plt.savefig('_output/figures/rec_moments.png', **savefig_kws)
plt.show()

In [ ]:
fig, axes = pplt.subplots(ncols=len(_frames), nrows=4, figsize=(7, 5), spany=False)
kws_sim = dict(color='black')
kws_rec = dict(color='red', marker='|', lw=0)

for j, frame in enumerate(_frames):
    
    axes[0, j].set_title('Turn {}'.format(turns[frame]))
    
    Sigmas_rec = Sigmas_rec_list[frame]  # [node, trial, Sigma]
    emittances_means, emittances_stds = [], []
    for k in range(Sigmas_rec.shape[0]):
        emittances_mean, emittances_std = mean_std(Sigmas_rec[k, :], ba.emittances)
        emittances_means.append(emittances_mean)
        emittances_stds.append(emittances_std)
    emittances_means = np.array(emittances_means)
    emittances_stds = np.array(emittances_stds)
    
    Sigmas_sim = Sigmas_sim_list[frame]
    emittances_sim = [ba.emittances(Sigma) for Sigma in Sigmas_sim]
    emittances_sim = np.array(emittances_sim)
    for k in range(4):
        ax = axes[k, j]
        ax.plot(rec_node_positions[:, 0], emittances_sim[:, k], **kws_sim)
        ax.errorbar(rec_node_positions[:, 0], emittances_means[:, k], 
                    yerr=emittances_stds[:, k],
                    elinewidth=1.0, ecolor='red', capsize=0, lw=0, zorder=999,
                   )
axes.format(xlabel='Position [m]', ylabel='[mm mrad]')
for ax, label in zip(axes[:, 0], [r"$\varepsilon_x$", r"$\varepsilon_y$", r"$\varepsilon_1$", r"$\varepsilon_2$"]):
    ax.set_ylabel('{} [mm mrad]'.format(label))
plt.show()

In [ ]:
# _frames = [1, 3, 5, 7]
# _cols = [1, 2, 3, 5, 6, 8]
# fig, axes = pplt.subplots(nrows=len(_cols), ncols=len(_frames), figsize=(7, 5.5), spany=False, aligny=True)
# for j, frame in enumerate(_frames):    
#     stats_rec = stats_rec_list[j]
#     stats_sim = stats_sim_list[j]
#     turn = turns[frame]
#     axes[0, j].set_title('Turn {}'.format(turn))
#     for i, col in enumerate(_cols):
#         ax = axes[i, j]
#         ax.plot(rec_node_positions[:, 0], stats_sim.corr.iloc[:, col].values, **sim_kws)
#         ax.plot(rec_node_positions[:, 0], stats_rec.corr.iloc[:, col].values, **rec_kws)
# for i, ax in enumerate(axes[:, 0]):
#     ax.set_ylabel(labels[i])
# xmin, xmax = axes[-1].get_xlim()
# axes.format(xlim=(xmin, 1.01 * xmax))
# axes.format(suptitle='Reconstructed correlation coefficients', xlabel='Position [m]')
# plt.savefig('_output/figures/rec_corr.png', **savefig_kws)
# plt.show()

## Load beam distribution

In [ ]:
import sys
sys.path.append('/Users/46h/Research/')
from scdist.tools import utils
from scdist.tools import plotting as myplt

In [ ]:
coords = utils.load_stacked_arrays('_input/coords_rtbt.npz')

In [ ]:
for i, X in enumerate(coords):
    turn = turns[i]
    axes = myplt.corner(X[:, :4], figsize=(5, 5), autolim_kws=dict(sigma=3.0), 
                        cmap=pplt.Colormap('mono', left=0.03), 
                        thresh=0,
                        bins='auto')
    plt.suptitle('Turn {}'.format(turn))